# Exploratory Analysis - DC Moving Violations, 2015-2020

This notebook looks at fine payments, moving violation counts and moving violation types across wards.

### Import Libraries

In [2]:
##Import Libraries
import psycopg2 #PostgreSQL driver
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas.io.sql as psql
import geopandas as gpd
import SQL_cred
from collections import Counter
print(SQL_cred.PGHOST)

dc-crash-bot-test.cw2qdhdq18cy.us-east-1.rds.amazonaws.com


### Create Ward Population Data Frame

In [3]:
#Info Pulled From https://www.dchealthmatters.org/demographicdata
dc_ward_pop=pd.DataFrame({'Population':[91673,92809,84979,87150,90380,103197,80669,80517],'ward_name':['Ward 1','Ward 2','Ward 3','Ward 4','Ward 5','Ward 6','Ward 7','Ward 8']})
dc_ward_pop

,Population,ward_name
0,91673,Ward 1
1,92809,Ward 2
2,84979,Ward 3
3,87150,Ward 4
4,90380,Ward 5
5,103197,Ward 6
6,80669,Ward 7
7,80517,Ward 8


## Fine Payment

### Load Data, Basic Handling

In [4]:
#Set SQL Address, Create SQL import function 'load_data'
conn_string = "host="+ SQL_cred.PGHOST +" port="+ "5432" +" dbname="+ SQL_cred.PGDATABASE +" user=" + SQL_cred.PGUSER \
+" password="+ SQL_cred.PGPASSWORD

def load_data(query):

    sql_command = query
    print (sql_command)

    # Load the data
    data = pd.read_sql(sql_command, conn)

    print(data.shape)
    return (data)

In [5]:
#Select all fine information from 2015 to 2020
conn=psycopg2.connect(conn_string)
query="SELECT fine_amount,total_paid, ward_name FROM analysis_data.moving_violations WHERE EXTRACT (year FROM issue_date) BETWEEN 2015 AND 2020;"
cursor = conn.cursor()
df=load_data(query)
df.to_csv('Fines.csv')

In [6]:
df=pd.read_csv('Fines.csv',index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'Fines.csv'

In [ ]:
df.fine_amount.fillna(0,inplace=True)
df.total_paid.fillna(0,inplace=True)
df.ward_name.fillna('Unknown',inplace=True)

pd.set_option('display.max_rows', None)
df.head(10)

In [ ]:
#Set Categories for fines & how they were paid
FvP=[]
for fine,paid in list(zip(df.fine_amount,df.total_paid)):
    if fine==0 and paid==0:
        FvP.append('No Fine')
    elif fine==paid:
        FvP.append('Fined & Paid Full')
    elif fine>0 and paid==0:
        FvP.append('Fined & Paid Nothing')
    elif fine>0 and fine>paid:
        FvP.append('Fined & Paid Less')
    elif fine>0 and fine<paid:
        FvP.append('Fined & Paid More')
    else:
        FvP.append('Na')
df['Fined_v_Paid']=FvP

### How fines were handled for violations in all of DC, 2015-2020

In [ ]:
df_fines_alldc=pd.concat([df['Fined_v_Paid'].value_counts(),(df['Fined_v_Paid'].value_counts()/df['Fined_v_Paid'].value_counts().sum())*100],axis=1)
df_fines_alldc.columns=['Count','Percentage']
df_fines_alldc

In [ ]:
#Group fine counts by ward and fine type
df_grouped_fine=df.groupby(['ward_name','Fined_v_Paid']).count().reset_index(level=[0,1])
df_fine_count=df_grouped_fine.pivot(index='ward_name',columns=['Fined_v_Paid'],values='fine_amount').drop(columns='Na')

In [ ]:
#Create a data frame with fine percentages by ward
df_fine_pct=df_fine_count.apply((lambda x: x/x.sum()),axis=1)
df_fine_count['Total']=df_fine_count.sum(axis=1)

### Counts of how fines were handled for traffic incidents by ward, sorted by total, 2015-2020

In [ ]:
df_fine_count.sort_values('Total',ascending=False)

### Percentages of how fines were handled for traffic incidents by ward, sorted by Paid Nothing, 2015-2020

In [ ]:
#df_fine_pct=df_fine_pct*100
df_fine_pct.sort_values('Fined & Paid Nothing', ascending=False)

#  Violation Count & Types by Ward

## Violation Counts by Ward

### Load Traffic Data

In [ ]:
#Query database, save csv to local files
#Select all violation descriptons from 2015 to 2020 along with road type & ward name
#conn=psycopg2.connect(conn_string)
#query="SELECT ward_name, violation_process_desc, dcfunctionalclass_desc FROM analysis_data.moving_violations WHERE EXTRACT (year FROM issue_date) BETWEEN 2015 AND 2020;"
#cursor = conn.cursor()
#df_violations=load_data(query)
#df_violations.rename(columns={"violation_process_desc":"Violations"},inplace=True)
#df_violations.to_csv('Violations_by_Ward_2015_2020')

In [ ]:
#Read in local csv data
df_violations=pd.read_csv('Violations_by_Ward_2015_2020.csv',usecols=[1,2,3])
df_violations.head()

### Load Map 

In [ ]:
#https://opendata.dc.gov/datasets/0ef47379cbae44e88267c01eaec2ff6e_31?geometry=-77.668%2C38.800%2C-76.361%2C38.987
#https://towardsdatascience.com/lets-make-a-map-using-geopandas-pandas-and-matplotlib-to-make-a-chloropleth-map-dddc31c1983d
b='DC_Ward_Map/Ward_from_2012.shp'
map_df=gpd.read_file(b)
map_df=map_df.sort_values('WARD_ID')
map_df.plot()
plt.show()

### Get total violation counts per ward, per capita

In [ ]:
ViolationsByward=df_violations.groupby('ward_name').count()
ViolationsByward.reset_index(inplace=True)
ViolationsByward

In [ ]:
ViolationsBywardPop=ViolationsByward.merge(dc_ward_pop)
ViolationsBywardPop['TotalViolationsPerCap']=ViolationsBywardPop.Violations/ViolationsBywardPop.Population
ViolationsBywardPop=ViolationsBywardPop.sort_values('TotalViolationsPerCap',ascending=False).drop(columns='dcfunctionalclass_desc')

In [ ]:
ViolationsBywardPop

In [ ]:
map_df=map_df.merge(ViolationsBywardPop[['ward_name','TotalViolationsPerCap']],left_on='NAME',right_on='ward_name')

In [ ]:
variable = "TotalViolationsPerCap"
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(16, 8))
map_df.plot(column=variable, cmap="Blues", linewidth=0.8, ax=ax, edgecolor='0.8',legend=True)
plt.title('Total Moving Violations per Capita 2015-2020')
plt.show()

In [ ]:
ViolationsBywardPop

##### Ward 5, 7 & 8 have the highest per capita violations - Ward 7 & 8 have at least 2.5x greater than Wards 4, 3, 6 & 1. 


##### But wards 5, 7 & 8 also have the most highways - presumably . Can the data be normalized by which types of roads they occur on?


### Determining the road type where violations occur most

In [ ]:
#Fill unknown road types with 'Unknown'
df_violations['dcfunctionalclass_desc']=df_violations['dcfunctionalclass_desc'].fillna('Unknown')
df_violations.head(5)

In [ ]:
#Group by ward & road type
ViolationsByRoadtype=df_violations.groupby(['ward_name','dcfunctionalclass_desc']).count()
ViolationsByRoadtype.reset_index(inplace=True)
ViolationsByRoadtype.head(5)

##### Looks like a lot of road types are unknown for these violations. Is this systematic across wards?

In [ ]:
#Look at just the unknown road types
ViolationsUnknownLocationType=ViolationsByRoadtype.loc[ViolationsByRoadtype['dcfunctionalclass_desc']=='Unknown']
ViolationsUnknownLocationType=ViolationsUnknownLocationType.merge(ViolationsBywardPop,on='ward_name',suffixes=['_Unkn','_Tot'])
ViolationsUnknownLocationType
#ViolationsByward2_['NormalizedViolations']=ViolationsByward2.Violations/ViolationsByward2.Population
#ViolationsByward2_.sort_values('NormalizedViolations',ascending=False).drop(columns='dcfunctionalclass_desc')

In [ ]:
ViolationsUnknownLocationType['Unknown_by_ward']=ViolationsUnknownLocationType.Violations_Unkn/ViolationsUnknownLocationType.Violations_Tot
ViolationsUnknownLocationType.sort_values('Unknown_by_ward',ascending=False)

##### Unfortunately, the number of Unknown road types is extremely spread out across wards - accounting for 95% of incidents for ward 3 and just 6 % for ward 4, with all the other roads scattered in between. We'll need to fill in more of these data points to normalize by this value.
##### Instead, the types of violations across wards can be investigated.

# Violation Types by Ward

In [ ]:
#Print all unique violation types
pd.set_option('display.max_rows', None)
df_violations.Violations.value_counts()

#### Separately, I downloaded these unique values into a csv file called 'Violation_Categories.csv', and assigned them one of the following categories:
##### D: Immediately Dangerous to Others 
Dangerous driving behavior that is obviously in violation of traffic laws e.g. speeding, improper turning
##### S: Secondary Conduct Violations  
Violations that are not obvious from driver behavior (e.g. no insurance) or pertain to the drivers vehicle (e.g. broken headlights)
##### MISC: Miscellaneous  
Commercial, truck, taxi violations, etc

#### These categories and their criteria stand to be refined. 

In [ ]:
#Read in categories csv
df_categories=pd.read_csv('Categories_csv.csv',index_col=1)
dict_cat=df_categories.to_dict()['Category']
#Map Categories
df_violations['Category']=df_violations.Violations.map(dict_cat)
dict2={'S':'Secondary Conduct Violations','D':'Immediately Dangerous to Others','MISC':'Miscellaneous'}
df_violations['Category']=df_violations['Category'].map(dict2)

In [ ]:
#Read in categories dict
#with open("Categories.json", "r") as infile:  
#    dict_=json.load(infile)
#df_violations2.rename(mapper={'dcfunctionalclass_desc':'Count'},inplace=True)
#df_violations['Category']=df_violations.Violations.map(dict_)

In [ ]:
#Create new df 'df_violations_' that groups by ward & type of violation
df_violations_=df_violations.drop(columns=['dcfunctionalclass_desc']).copy()
df_violations_=df_violations_.groupby(['ward_name','Category']).count()
df_violations_.reset_index(level=[0,1],inplace=True)
df_violations_.head()

In [ ]:
#Merge with population data, determine violation types per capita
ViolationsByward_=df_violations_.merge(dc_ward_pop,on='ward_name')
ViolationsByward_['ViolationsPerCapita']=ViolationsByward_.Violations/ViolationsByward_.Population
ViolationsByward_.sort_values('ViolationsPerCapita',ascending=False)

In [ ]:
#Pivot data frame: ward is index, columns are type of violation, values are violation per capita
ViolationsByward_pivot=ViolationsByward_.pivot(index='ward_name',columns=['Category'],values='ViolationsPerCapita')
ViolationsByward_pivot.reset_index(inplace=True)
ViolationsByward_pivot

In [ ]:
#Merge Map dataframe with Violation Type per Capita Data
map_df=map_df.merge(ViolationsByward_pivot,left_on='NAME',right_on='ward_name')

In [ ]:
#Map per capita incidence of "Immediately Dangerous to Others" violations
variable = "Immediately Dangerous to Others"
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(16, 8))
map_df.plot(column=variable, cmap="Reds", linewidth=0.8, ax=ax, edgecolor='0.8',legend=True)
plt.title('Immediately Dangerous to Others Traffic Violations Per Capita by Ward 2015-2020')
plt.show()

In [ ]:
#Map per capita incidence of ""Secondary Conduct Violations" violations
variable = "Secondary Conduct Violations"
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(16, 8))
map_df.plot(column=variable, cmap="Greens", linewidth=0.8, ax=ax, edgecolor='0.8',legend=True)
plt.title('Secondary Conduct Violations Per Capita by Ward 2015-2020')
plt.show()

In [ ]:
#Map per capita incidence of "Miscellaneous" violations
variable = "Miscellaneous"
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(16, 8))
map_df.plot(column=variable, cmap="Purples", linewidth=0.8, ax=ax, edgecolor='0.8',legend=True)
plt.title('Miscellaneous 2015-2020')
plt.show()

## Word Vectorizer Code - Ignore

In [ ]:
Speeding=['MPH','SPEEDING','SPEED']
Traffic=['STOP','DISTRACTED','ATTENTION','TURN','LEFT','RIGHT','LANES','STREET','YIELD','TRAFFIC','PASS','DRIVE','DRIVING','INTERSECTION','PEDESTRIAN','CYCLIST','BICYCLE']
Permit=['REGISTRATION','REGISTERED','UNREGISTERED','INSURANCE','PERMIT','TAGS']
Car=['WINDOW','WINDOWS','LIGHT','LIGHTS','PLATE','BUMPER','BUMPERS','PLATES']

In [ ]:
k=df_violations.Violations
p=[i.lower() for i in k[0:600000]]

In [ ]:
Violations=pd.DataFrame(df_violations.Violations.unique(),columns=['Violations'])

In [ ]:
Violations.dropna(axis=0)

In [ ]:
#Word Preprocessing
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')

#Make a dataframe of unique column names
Violations=pd.DataFrame(df_violations.Violations.unique(),columns=['Violations'])
#Remove 'None' Entry
Violations.dropna(axis=0,inplace=True)
#Replace any non letters with a white space (the carrot makes it 'replace everything except')
Violations['Violations_Edit']= Violations['Violations'].str.replace('[^a-zA-Z]', ' ')
#Make all text lower case
Violations['Violations_Edit'] = Violations['Violations_Edit'].str.lower()

In [ ]:
Violations

In [ ]:
#Correct Misspellings
Violations['Violations_Edit'].loc[1]='owner operate or permit operation of uninsured vehicle'
Violations['Violations_Edit'].loc[176]='less than yrs old mandatory seat belt violation'
Violations['Violations_Edit'].loc[214]='less than yrs old passenger restraint violation'
Violations['Violations_Edit'].loc[114]='bike pmd fail to yield right of way to pedestrian or vehicle'
Violations['Violations_Edit'].loc[254]='failure yield row transit bus'

#Lemmatize Entires, Remove Stop Words
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

def lemmatize_remove_stopwords(text):
    y=[lemmatizer.lemmatize(w) for w in word_tokenize(text)]
    h=[c for c in y if c not in stopwords.words('english')]
    return " ".join(h)

Violations['Violations_Edit']=Violations['Violations_Edit'].apply(lemmatize_remove_stopwords)
Violations

In [ ]:
Violations.to_csv('Violations_Unique.csv')

In [ ]:
h=Violations['Violations_Edit'].tolist()
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(h)
pd.set_option('display.max_rows', None)
df_vw=pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names())
df_vw.T

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=5)
model.fit(X_train_counts)
components_df = pd.DataFrame(model.components_, columns=df_vw.columns)
for i in range(0,5):
    component = components_df.iloc[i]
    print('Component ',i)
    print(component.nlargest())

In [ ]:
transformed_data=model.transform(X_train_counts)
df_=pd.DataFrame(transformed_data,index=Violations.Violations)
#df_.columns=['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine']
df_.columns=['Zero','One','Two','Three','Four']
df_['MajorComponent']=df_.idxmax(axis=1).values
df_.sort_values('MajorComponent')
category_map={'Zero':'Dangerous to Others','One':'Conduct Violations','Two':'Dangerous to Others','Three':'Dangerous to Others','Four':'Conduct Violations',}
df_['New_Map']=df_['MajorComponent'].map(category_map)
df_.reset_index(inplace=True)

In [ ]:
from sklearn.preprocessing import normalize
norm_features = normalize(transformed_data)
df = pd.DataFrame(norm_features,index=Violations.Violations)
article = df.iloc[81]
# Compute the dot products: similarities
similarities = df.dot(article)
print(similarities.nlargest())

In [ ]:
#Code for generating categories via input
#dict_={}
#for i in list(df_['Violations']):
#    txt=input(i+' ')
#    dict_[i]=txt
#dict_['OPERATE A VEHICLE IN VIOLATION OF A RESTRICTION']=C
#dict_['SPEEDING IN CMV UP TO 10 MPH OVER SPEED LIMIT C']=D
#dict_['FAIL TO TURN WHEEL TO CURB']=D
#import json
#with open("Categories.json", "w") as outfile:  
#    json.dump(dict_, outfile) 